# Bias Injection Experiments

This notebook creates synthetic biased users with strong preferences for Adventure or Mystery genres
and trains SVD models to study bias propagation effects.

In [5]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

## 1. Load Base Data

In [6]:
# Load processed data
ratings = pd.read_csv('../data/processed/clean_ratings.csv')
books = pd.read_csv('../data/processed/clean_books.csv')
adventure_book_ids = pd.read_csv('../data/processed/adventure_book_ids.csv')['book_id'].tolist()
mystery_book_ids = pd.read_csv('../data/processed/mystery_book_ids.csv')['book_id'].tolist()

# Load baseline results for comparison
baseline_results = pd.read_csv('../results/baseline/baseline_results.csv').iloc[0]
with open('../results/baseline/baseline_testset.pkl', 'rb') as f:
    baseline_testset = pickle.load(f)

print(f"📚 Original dataset: {len(ratings):,} ratings")
print(f"🗺️  Adventure books: {len(adventure_book_ids)}")
print(f"🔍 Mystery books: {len(mystery_book_ids)}")
print(f"🎯 Baseline RMSE: {baseline_results['overall_rmse']:.4f}")

📚 Original dataset: 5,976,479 ratings
🗺️  Adventure books: 3525
🔍 Mystery books: 4084
🎯 Baseline RMSE: 0.8310


## 2. Create Synthetic Biased Users

In [7]:
def create_biased_users(genre_book_ids, num_users, genre_name, min_ratings=20, max_ratings=50):
    """
    Create synthetic users with strong bias toward a specific genre
    
    Args:
        genre_book_ids: List of book IDs for the target genre
        num_users: Number of biased users to create
        genre_name: Name of the genre (for logging)
        min_ratings: Minimum number of ratings per user
        max_ratings: Maximum number of ratings per user
    
    Returns:
        DataFrame with synthetic ratings
    """
    print(f"🔄 Creating {num_users} {genre_name}-biased users...")
    
    # Get next available user ID
    max_user_id = ratings['user_id'].max()
    
    synthetic_ratings = []
    
    for i in range(num_users):
        user_id = max_user_id + i + 1
        num_ratings = np.random.randint(min_ratings, max_ratings + 1)
        
        # Sample books for this user
        user_books = np.random.choice(genre_book_ids, 
                                     min(num_ratings, len(genre_book_ids)), 
                                     replace=False)
        
        for book_id in user_books:
            # Biased users give high ratings (4-5) with high probability
            rating = np.random.choice([3, 4, 5], p=[0.1, 0.4, 0.5])
            
            synthetic_ratings.append({
                'user_id': user_id,
                'book_id': book_id,
                'rating': rating
            })
    
    synthetic_df = pd.DataFrame(synthetic_ratings)
    print(f"✅ Created {len(synthetic_df)} synthetic ratings for {num_users} users")
    return synthetic_df

# Define experiment parameters
bias_levels = [50, 100, 200, 500, 1000, 2000]  # Number of biased users to inject
genres = {
    'adventure': adventure_book_ids,
    'mystery': mystery_book_ids
}

print(f"📋 Will test bias levels: {bias_levels}")
print(f"📋 Will test genres: {list(genres.keys())}")

📋 Will test bias levels: [50, 100, 200, 500, 1000, 2000]
📋 Will test genres: ['adventure', 'mystery']


## 3. Run Bias Injection Experiments

In [8]:
# Function to evaluate model with bias injection
def evaluate_biased_model(original_ratings, synthetic_ratings, testset, genre_name, num_biased_users):
    """
    Train model with bias injection and evaluate
    """
    # Combine original training data with synthetic biased users
    combined_ratings = pd.concat([original_ratings, synthetic_ratings], ignore_index=True)
    
    # Create Surprise dataset
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(combined_ratings[['user_id', 'book_id', 'rating']], reader)
    trainset = data.build_full_trainset()
    
    # Train SVD model
    svd_model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
    svd_model.fit(trainset)
    
    # Evaluate on the same test set as baseline
    predictions = svd_model.test(testset)
    
    # Calculate overall metrics
    rmse = accuracy.rmse(predictions, verbose=False)
    mae = accuracy.mae(predictions, verbose=False)
    
    # Calculate genre-specific metrics
    adventure_predictions = [p for p in predictions if p.iid in adventure_book_ids]
    mystery_predictions = [p for p in predictions if p.iid in mystery_book_ids]
    other_predictions = [p for p in predictions if p.iid not in adventure_book_ids + mystery_book_ids]
    
    def calc_metrics(pred_list):
        if not pred_list:
            return None, None
        errors = [abs(p.est - p.r_ui) for p in pred_list]
        squared_errors = [(p.est - p.r_ui) ** 2 for p in pred_list]
        return np.mean(errors), np.sqrt(np.mean(squared_errors))
    
    adv_mae, adv_rmse = calc_metrics(adventure_predictions)
    mys_mae, mys_rmse = calc_metrics(mystery_predictions)
    oth_mae, oth_rmse = calc_metrics(other_predictions)
    
    # Generate sample recommendations to check bias propagation
    sample_recs = generate_sample_recommendations(svd_model, trainset, n_users=50, n_recs=10)
    genre_dist = analyze_recommendation_bias(sample_recs, adventure_book_ids, mystery_book_ids)
    
    return {
        'genre': genre_name,
        'num_biased_users': num_biased_users,
        'total_ratings': len(combined_ratings),
        'synthetic_ratings': len(synthetic_ratings),
        'overall_rmse': rmse,
        'overall_mae': mae,
        'adventure_mae': adv_mae,
        'adventure_rmse': adv_rmse,
        'mystery_mae': mys_mae,
        'mystery_rmse': mys_rmse,
        'other_mae': oth_mae,
        'other_rmse': oth_rmse,
        'rec_adventure_pct': genre_dist['adventure_pct'],
        'rec_mystery_pct': genre_dist['mystery_pct'],
        'rec_other_pct': genre_dist['other_pct']
    }

def generate_sample_recommendations(model, trainset, n_users=50, n_recs=10):
    """Generate sample recommendations for bias analysis"""
    all_book_ids = set([iid for (uid, iid, rating) in trainset.all_ratings()])
    all_user_ids = set([uid for (uid, iid, rating) in trainset.all_ratings()])
    
    # Sample original users only (exclude synthetic users)
    original_users = [uid for uid in all_user_ids if uid <= ratings['user_id'].max()]
    sample_user_ids = np.random.choice(original_users, min(n_users, len(original_users)), replace=False)
    
    recommendations = []
    
    for uid in sample_user_ids:
        user_rated_books = set([iid for (u, iid, rating) in trainset.all_ratings() if u == uid])
        candidates = list(all_book_ids - user_rated_books)
        
        if len(candidates) < n_recs:
            continue
            
        user_predictions = [(iid, model.predict(uid, iid).est) for iid in candidates]
        user_predictions.sort(key=lambda x: x[1], reverse=True)
        
        for rank, (book_id, rating) in enumerate(user_predictions[:n_recs], 1):
            recommendations.append((uid, book_id, rating, rank))
    
    return recommendations

def analyze_recommendation_bias(recommendations, adventure_ids, mystery_ids):
    """Analyze genre distribution in recommendations"""
    adventure_count = sum(1 for _, book_id, _, _ in recommendations if book_id in adventure_ids)
    mystery_count = sum(1 for _, book_id, _, _ in recommendations if book_id in mystery_ids)
    other_count = len(recommendations) - adventure_count - mystery_count
    total = len(recommendations)
    
    return {
        'adventure_pct': (adventure_count / total * 100) if total > 0 else 0,
        'mystery_pct': (mystery_count / total * 100) if total > 0 else 0,
        'other_pct': (other_count / total * 100) if total > 0 else 0
    }

## 4. Execute Experiments

In [ ]:
# Prepare original training data (exclude test set)
# We need to recreate the same train-test split as baseline
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)
trainset, _ = train_test_split(data, test_size=0.2, random_state=42)

# Convert trainset back to DataFrame for easier manipulation
original_train_data = []
for uid, iid, rating in trainset.all_ratings():
    original_train_data.append({'user_id': uid, 'book_id': iid, 'rating': rating})
original_train_df = pd.DataFrame(original_train_data)

print(f"📚 Original training data: {len(original_train_df):,} ratings")
print(f"🧪 Test data: {len(baseline_testset):,} ratings")

# Run all experiments
all_results = []

# Add baseline results for comparison
baseline_row = {
    'genre': 'baseline',
    'num_biased_users': 0,
    'total_ratings': len(original_train_df),
    'synthetic_ratings': 0,
    'overall_rmse': baseline_results['overall_rmse'],
    'overall_mae': baseline_results['overall_mae'],
    'adventure_mae': baseline_results['adventure_mae'],
    'adventure_rmse': baseline_results['adventure_rmse'],
    'mystery_mae': baseline_results['mystery_mae'],
    'mystery_rmse': baseline_results['mystery_rmse'],
    'other_mae': baseline_results['other_mae'],
    'other_rmse': baseline_results['other_rmse'],
    'rec_adventure_pct': baseline_results['adventure_rec_percentage'],
    'rec_mystery_pct': baseline_results['mystery_rec_percentage'],
    'rec_other_pct': baseline_results['other_rec_percentage']
}
all_results.append(baseline_row)

# Run bias injection experiments
total_experiments = len(genres) * len(bias_levels)
experiment_count = 0

for genre_name, genre_book_ids in genres.items():
    for num_biased_users in bias_levels:
        experiment_count += 1
        print(f"\n🔬 Experiment {experiment_count}/{total_experiments}: {genre_name} bias with {num_biased_users} users")
        
        # Create synthetic biased users
        synthetic_ratings = create_biased_users(genre_book_ids, num_biased_users, genre_name)
        
        # Train and evaluate model
        results = evaluate_biased_model(original_train_df, synthetic_ratings, 
                                      baseline_testset, genre_name, num_biased_users)
        
        all_results.append(results)
        
        print(f"   📊 RMSE: {results['overall_rmse']:.4f} (Δ: {results['overall_rmse'] - baseline_results['overall_rmse']:+.4f})")
        print(f"   🗺️  Adventure recs: {results['rec_adventure_pct']:.1f}% (Δ: {results['rec_adventure_pct'] - baseline_results['adventure_rec_percentage']:+.1f}%)")
        print(f"   🔍 Mystery recs: {results['rec_mystery_pct']:.1f}% (Δ: {results['rec_mystery_pct'] - baseline_results['mystery_rec_percentage']:+.1f}%)")

print("\n✅ All experiments completed!")

📚 Original training data: 4,781,183 ratings
🧪 Test data: 1,195,296 ratings

🔬 Experiment 1/12: adventure bias with 50 users
🔄 Creating 50 adventure-biased users...
✅ Created 1828 synthetic ratings for 50 users
   📊 RMSE: 1.1289 (Δ: +0.2978)
   🗺️  Adventure recs: 42.4% (Δ: -6.4%)
   🔍 Mystery recs: 33.8% (Δ: +25.6%)

🔬 Experiment 2/12: adventure bias with 100 users
🔄 Creating 100 adventure-biased users...
✅ Created 3520 synthetic ratings for 100 users
   📊 RMSE: 1.1290 (Δ: +0.2980)
   🗺️  Adventure recs: 38.6% (Δ: -10.2%)
   🔍 Mystery recs: 34.0% (Δ: +25.8%)

🔬 Experiment 3/12: adventure bias with 200 users
🔄 Creating 200 adventure-biased users...
✅ Created 7042 synthetic ratings for 200 users
   📊 RMSE: 1.1290 (Δ: +0.2980)
   🗺️  Adventure recs: 40.0% (Δ: -8.8%)
   🔍 Mystery recs: 36.0% (Δ: +27.8%)

🔬 Experiment 4/12: adventure bias with 500 users
🔄 Creating 500 adventure-biased users...
✅ Created 17538 synthetic ratings for 500 users
   📊 RMSE: 1.1284 (Δ: +0.2974)
   🗺️  Adventure re

## 5. Save Results

In [ ]:
# Save all results
results_df = pd.DataFrame(all_results)
results_df.to_csv('../results/biased/bias_injection_results.csv', index=False)

print("✅ Results saved!")
print(f"📁 Results saved to: ../results/biased/bias_injection_results.csv")
print(f"📊 Total experiments: {len(results_df)}")
print("\n🔄 Next step: Run 04_bias_analysis.ipynb to analyze results")